# Anatomy of a QualibrationNode

This guide dissects a typical QUAlibrate calibration node, using the `02a_resonator_spectroscopy.py` script as a concrete example. We will go through the script section by section, presenting the code first, followed by a detailed explanation of its purpose and functionality within the node's workflow.

All calibration node scripts follow a standardized structure, segmented into distinct sections using `# %% {Section Name}` separators. This consistent structure makes nodes easier to understand, maintain, and integrate. This segmentation also allows developers to run individual cells interactively within a Python kernel (e.g., in VS Code), which is highly beneficial for development and debugging.

The common sections found in most calibration nodes are:
* `Imports`: Handles all necessary library and module imports.
* `Initialisation`: Defines the node, description, parameters, and loads the QUAM state.
* `Create_QUA_program`: Defines the core QUA pulse sequence for the experiment.
* `Simulate` (Optional): Simulates the QUA program's waveforms and timing.
* `Execute`: Runs the QUA program on the quantum hardware and fetches results.
* `Load_data` (Optional): Loads data from a previous run instead of executing.
* `Analyse_data`: Processes the raw data and performs fitting or analysis.
* `Plot_data`: Generates plots visualizing the data and analysis results.
* `Update_state`: Proposes modifications to the QUAM state based on results.
* `Save_results`: Persists all parameters, results, figures, and proposed state updates.

Each of these common sections typically corresponds to a Python function decorated with `@node.run_action`. This decorator effectively encapsulates the logic for that specific step (like creating the QUA program or analysing data) and registers it as an action associated with the node instance, much like a method belongs to a class object. This approach promotes modularity and allows the QUAlibrate framework to manage the execution flow, including conditional skipping of steps.

## Imports Section

In [ ]:
# %% {Imports}
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from dataclasses import asdict

from qm.qua import *

from qualang_tools.loops import from_array
from qualang_tools.multi_user import qm_session
from qualang_tools.results import progress_counter
from qualang_tools.units import unit

from qualibrate import QualibrationNode
from quam_config import Quam
from calibration_utils.resonator_spectroscopy import (
    Parameters,
    process_raw_dataset,
    fit_raw_data,
    log_fitted_results,
    plot_raw_amplitude_with_fit,
    plot_raw_phase,
)
from qualibration_libs.parameters import get_qubits
from qualibration_libs.runtime import simulate_and_plot
from qualibration_libs.data import XarrayDataFetcher

### Explanation: Imports Section

This initial section handles importing all necessary libraries and modules.
- **Standard Libraries:** `matplotlib.pyplot` for plotting, `numpy` for numerical operations, `xarray` for handling multi-dimensional labeled data (often used for results), and `asdict` from `dataclasses` for converting data objects to dictionaries.
- **QM Libraries:** Imports everything from `qm.qua` (like `program`, `declare`, `measure`, `play`, QUA control flow statements like `for_`, `save`, `stream_processing`, etc.).
- **Qualang Tools:** Imports specific utilities like `from_array` (for easily iterating over numpy arrays in QUA), `qm_session` (a context manager for handling Quantum Machine execution, preventing conflicts), `progress_counter` (for displaying progress during data acquisition), and `unit` (for handling physical units and conversions).
- **QUAlibrate/QUAM Core:** Imports `QualibrationNode` (the core class for creating calibration nodes) and `logger` from `qualibrate`, and the `Quam` class itself from the `quam_config` package.
- **Experiment-Specific Imports:** Imports the `Parameters` class specific to this resonator spectroscopy experiment, along with functions for processing (`process_raw_dataset`), fitting (`fit_raw_data`), logging (`log_fitted_results`), and plotting (`plot_raw_amplitude_with_fit`, `plot_raw_phase`) from the corresponding `qualibration_utils.resonator_spectroscopy` package.
- **Workflow/Helper Imports:** The package `qualibration_libs` contains a series of tools that are generally useful for calibration nodes, as opposed to being useful for a specific node. This includes `get_qubits` (a helper to determine which qubits to target based on parameters), `simulate_and_plot` (a workflow utility for running simulations), and `XarrayDataFetcher` (a tool for fetching results from QUA jobs directly into xarray Datasets).

## Initialisation Section

In [ ]:
# %% {Initialisation}
description = """
        1D RESONATOR SPECTROSCOPY
This sequence involves measuring the resonator by sending a readout pulse and demodulating the signals to extract the
'I' and 'Q' quadratures across varying readout intermediate frequencies for all the active qubits.
The data is then post-processed to determine the resonator resonance frequency.
This frequency is used to update the readout frequency in the state.

Prerequisites:
    - Having calibrated the IQ mixer/Octave connected to the readout line (node 01a_mixer_calibration.py).
    - Having calibrated the time of flight, offsets, and gains (node 01a_time_of_flight.py).
    - Having initialized the QUAM state parameters for the readout pulse amplitude and duration, and the resonators depletion time.
    - Having specified the desired flux point if relevant (qubit.z.flux_point).

State update:
    - The readout frequency: qubit.resonator.f_01 & qubit.resonator.RF_frequency
"""

# Be sure to include [Parameters, Quam] so the node has proper type hinting
node = QualibrationNode[Parameters, Quam](
    name="02a_resonator_spectroscopy",  # Name should be unique
    description=description,  # Describe what the node is doing, which is also reflected in the QUAlibrate GUI
    parameters=Parameters(),  # Node parameters defined under quam_experiment/experiments/node_name
)


# Any parameters that should change for debugging purposes only should go in here
# These parameters are ignored when run through the GUI or as part of a graph
@node.run_action(skip_if=node.modes.external)
def custom_param(node: QualibrationNode[Parameters, Quam]):
    """Allow the user to locally set the node parameters for debugging purposes, or execution in the Python IDE."""
    # You can get type hinting in your IDE by typing node.parameters.
    node.parameters.qubits = ["q1", "q2"]
    pass


# Instantiate the QUAM class from the state file
node.machine = Quam.load()

### Explanation: Initialisation Section

This section sets up the core `QualibrationNode` object and loads the machine state (QUAM).
- **Description String:** A multi-line string `description` is defined. This provides a human-readable explanation of the node's purpose, prerequisites, and the parameters it intends to update. This description is displayed in the QUAlibrate user interface.
- **Node Instantiation:** An instance of `QualibrationNode` is created.
    - `QualibrationNode[Parameters, Quam]`: Type hints are used to associate the node with the specific `Parameters` class (imported earlier) and the `Quam` class. This aids IDEs with type checking and autocompletion.
    - `name="02a_resonator_spectroscopy"`: Assigns a unique identifier to this node type. This name is crucial for QUAlibrate to track and manage the node.
    - `description=description`: Passes the description string defined above.
    - `parameters=Parameters()`: Passes an *instance* of the imported `Parameters` class. QUAlibrate uses this to manage the node's input parameters (reading defaults, accepting user input via the UI).
- **Custom Parameter Run Action (`custom_param`):**
    - `@node.run_action(skip_if=node.modes.external)`: Defines a function `custom_param` as a run action. The `skip_if=node.modes.external` argument ensures this function *only* runs when the script is executed directly (standalone mode, e.g., in an IDE) and is *skipped* when run via the QUAlibrate UI or as part of a graph.
    - **Purpose:** This action allows developers to temporarily override parameters (like `node.parameters.qubits = ["q1", "q2"]`) for local debugging or testing without affecting the default parameters used in automated runs.
- **QUAM Loading:**
    - `node.machine = Quam.load()`: Loads the state of the quantum machine from the configuration file specified within the QUAM parameters (often defined in a base `Parameters` class). The loaded `Quam` object is stored in `node.machine`, making the entire machine configuration accessible to subsequent run actions.

## Create QUA Program Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is not None)
def create_qua_program(node: QualibrationNode[Parameters, Quam]):
    """Create the sweep axes and generate the QUA program from the pulse sequence and the node parameters."""
    # Initialize unit conversion tool
    u = unit(coerce_to_integer=True)
    # Determine target qubits and prepare for potential batching
    node.namespace["qubits"] = qubits = get_qubits(node)
    num_qubits = len(qubits)
    # Calculate the frequency sweep vector (detunings) based on parameters
    n_avg = node.parameters.num_averages
    span = node.parameters.frequency_span_in_mhz * u.MHz
    step = node.parameters.frequency_step_in_mhz * u.MHz
    dfs = np.arange(-span / 2, +span / 2, step)
    # Store sweep axes metadata for data fetching/labeling
    node.namespace["sweep_axes"] = {
        "qubit": xr.DataArray(qubits.get_names()),
        "detuning": xr.DataArray(dfs, attrs={"long_name": "readout frequency", "units": "Hz"}),
    }

    # Define the QUA program
    with program() as node.namespace["qua_program"]:
        # Declare QUA variables and streams using QUAM helpers for consistency
        I, I_st, Q, Q_st, n, n_st = node.machine.qua_declaration()
        df = declare(int)  # QUA variable for frequency detuning sweep

        # Loop over batches of qubits that can be measured in parallel
        for multiplexed_qubits in qubits.batch():
            # Set initial QPU state (e.g., flux bias) for the batch
            for qubit in multiplexed_qubits.values():
                node.machine.initialize_qpu(target=qubit)
            align()  # Ensure QPU initialization is complete before proceeding

            # Averaging loop
            with for_(n, 0, n < n_avg, n + 1):
                save(n, n_st)  # Save averaging progress counter

                # Frequency sweep loop (iterating through the 'dfs' numpy array)
                with for_(*from_array(df, dfs)):
                    # Loop through qubits within the current measurement batch
                    for i, qubit in multiplexed_qubits.items():
                        rr = qubit.resonator  # Get the resonator object from QUAM
                        # Dynamically update the resonator's frequency for this sweep point
                        rr.update_frequency(df + rr.intermediate_frequency)
                        # Perform the measurement (pulse defined in QUAM)
                        rr.measure("readout", qua_vars=(I[i], Q[i]))
                        # Wait for resonator cooldown (time defined in QUAM)
                        rr.wait(rr.depletion_time * u.ns)
                        # Save the raw I/Q data to streams
                        save(I[i], I_st[i])
                        save(Q[i], Q_st[i])
                    align()  # Ensure all qubits in the batch complete before next frequency step

        # --- Stream Processing ---
        # Executed by the QOP in real time after the loops finish
        with stream_processing():
            n_st.save("n")  # Save the final value of the averaging counter
            # Average the raw I/Q data over all averages for each qubit and frequency point
            for i in range(num_qubits):
                I_st[i].buffer(len(dfs)).average().save(f"I{i + 1}")  # Save averaged I vector
                Q_st[i].buffer(len(dfs)).average().save(f"Q{i + 1}")  # Save averaged Q vector

### Explanation: Create_QUA_program Section

This run action defines the core QUA pulse sequence for performing resonator spectroscopy. Its main goal is to generate a QUA program that sweeps the readout frequency around the expected resonance for each target qubit and measures the corresponding I/Q response, averaging the results for noise reduction.

- **Preparations:** The action first defines relevant variables for the QUA probram. 
  Relevant sweeps are collected into `node.namespace["sweep_axes"]`, to be used within the QUA program, and also by the `XarrayDataFetcher` to automatically associate the correct coordinate axes to the fetched data.
- **Program Definition** It then enters the `with program()` context to define the QUA sequence. Inside, it declares necessary QUA variables and streams, often using helper methods from the loaded `Quam` object (`node.machine`) for consistency.
- **Nested Loops:** The core logic involves nested loops:
    1.  An outer loop (`for multiplexed_qubits in qubits.batch()`) iterates through groups of qubits that can be measured in parallel.
    2.  An averaging loop (`with for_(n,...)`) repeats the measurement `n_avg` times.
    3.  A frequency sweep loop (`with for_(*from_array(df, dfs))`) iterates through the calculated frequency detuning values.
    4.  An inner loop (`for i, qubit in multiplexed_qubits.items()`) handles operations for each qubit within the current parallel batch.
- **Measurement Sequence:** Inside the innermost loop, for each qubit and frequency point:
    1.  The resonator's frequency is updated (`rr.update_frequency(...)`).
    2.  A measurement pulse is played, and the result demodulated (`rr.measure("readout", ...)`).
    3.  A wait time allows the resonator to relax (`rr.wait(...)`).
    4.  The raw I/Q results are saved to streams (`save(I[i], ...)`).
- **QUAM Integration:** The code heavily relies on the `Quam` object (`node.machine` and `qubit.resonator`) to get parameters like pulse names ("readout"), element properties (intermediate frequency, depletion time), and to use helper methods for QUA declaration, frequency updates, measurements, and waits.
- **Stream Processing:** After the loops complete, a `with stream_processing()` block defines how the QOP should process the raw data streams. Here, it averages the I and Q values collected over all `n_avg` iterations for each frequency point and saves the resulting averaged vectors (one I vector and one Q vector per qubit).
- **Output:** The generated QUA program is stored in `node.namespace["qua_program"]`, and metadata about the sweep axes is stored in `node.namespace["sweep_axes"]` for use in subsequent run actions (like execution and data fetching).

## Simulate Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is not None or not node.parameters.simulate)
def simulate_qua_program(node: QualibrationNode[Parameters, Quam]):
    """Connect to the QOP and simulate the QUA program"""
    # Connect to the QOP
    qmm = node.machine.connect()
    # Get the config from the machine
    config = node.machine.generate_config()
    # Simulate the QUA program, generate the waveform report and plot the simulated samples
    samples, fig, wf_report = simulate_and_plot(qmm, config, node.namespace["qua_program"], node.parameters)
    # Store the figure, waveform report and simulated samples
    node.results["simulation"] = {"figure": fig, "wf_report": wf_report, "samples": samples}

### Explanation: Simulate Section

This optional run action allows simulating the QUA program before running it on actual hardware.
- **Decorator:** `@node.run_action(skip_if=node.parameters.load_data_id is not None or not node.parameters.simulate)` ensures this action only runs if data isn't being loaded (`load_data_id` is None) AND the `simulate` parameter is set to `True`.
- **Connection & Config:**
    - `qmm = node.machine.connect()`: Uses a QUAM helper method to establish a connection to the Quantum Machines Manager (QMM).
    - `config = node.machine.generate_config()`: Uses a QUAM helper method to generate the hardware configuration dictionary based on the current QUAM state.
- **Simulation Execution:**
    - `samples, fig, wf_report = simulate_and_plot(...)`: Calls a workflow utility function (`simulate_and_plot` from `quam_experiments.workflow`) which likely wraps the standard `qmm.simulate` call. It passes the QMM object, the generated config, the QUA program (stored in `node.namespace["qua_program"]`), and the node parameters. This function typically returns the simulated waveforms (`samples`), a figure visualizing them (`fig`), and potentially a waveform report (`wf_report`).
- **Result Storage:**
    - `node.results["simulation"] = {"figure": fig, "wf_report": wf_report, "samples": samples}`: Stores the outputs of the simulation (figure, report, samples) in the `node.results` dictionary under the key "simulation".

## Execute Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is not None or node.parameters.simulate)
def execute_qua_program(node: QualibrationNode[Parameters, Quam]):
    """Connect to the QOP, execute the QUA program and fetch the raw data and store it in a xarray dataset called "ds_raw"."""
    # Connect to the QOP
    qmm = node.machine.connect()
    # Get the config from the machine
    config = node.machine.generate_config()
    # Execute the QUA program only if the quantum machine is available (this is to avoid interrupting running jobs).
    with qm_session(qmm, config, timeout=node.parameters.timeout) as qm:
        # The job is stored in the node namespace to be reused in the fetching_data run_action
        node.namespace["job"] = job = qm.execute(node.namespace["qua_program"])
        # Display the progress bar
        data_fetcher = XarrayDataFetcher(job, node.namespace["sweep_axes"])
        for dataset in data_fetcher:
            # print_progress_bar(job, iteration_variable="n", total_number_of_iterations=node.parameters.num_averages)
            progress_counter(
                data_fetcher["n"],
                node.parameters.num_averages,
                start_time=data_fetcher.t_start,
            )
        # Display the execution report to expose possible runtime errors
        print(job.execution_report())
    # Register the raw dataset
    node.results["ds_raw"] = dataset

### Explanation: Execute Section

This run action executes the QUA program on the actual hardware (QOP) and fetches the results.
- **Decorator:** `@node.run_action(skip_if=node.parameters.load_data_id is not None or node.parameters.simulate)` ensures this action only runs if data isn't being loaded AND simulation is not enabled.
- **Connection & Config:** Similar to the simulate action, it connects to the QMM (`node.machine.connect()`) and generates the hardware config (`node.machine.generate_config()`) using QUAM helpers.
- **Execution Context:**
    - `with qm_session(qmm, config, timeout=node.parameters.timeout) as qm:`: Uses the `qm_session` context manager (from `qualang_tools.multi_user`) to safely acquire and release the Quantum Machine (`qm`). This prevents multiple scripts from trying to use the same QOP simultaneously. A timeout parameter can be provided.
- **Job Execution:**
    - `node.namespace["job"] = job = qm.execute(node.namespace["qua_program"])`: Executes the QUA program stored in the namespace and stores the returned `job` object in the namespace as well.
- **Data Fetching & Progress:**
    - `data_fetcher = XarrayDataFetcher(job, node.namespace["sweep_axes"])`: Initializes the `XarrayDataFetcher` tool, passing it the running `job` and the `sweep_axes` metadata defined earlier. This tool handles fetching data from the QOP streams and organizing it into an xarray Dataset with correctly labeled dimensions.
    - `for dataset in data_fetcher:`: Iterates while the job is running. The `XarrayDataFetcher` yields intermediate or final datasets as data becomes available.
    - For more details about the `XarrayDataFetcher`, see the documentation at [https://github.com/qua-platform/qualibration-libs](https://github.com/qua-platform/qualibration-libs)
    - `progress_counter(...)`: Calls a utility to display a progress bar in the console based on the fetched value of the averaging counter (`data_fetcher["n"]`) compared to the total number of averages (`node.parameters.num_averages`).
- **Execution Report:** `print(job.execution_report())`: After the job finishes, prints a report containing information about the execution, including any potential runtime errors.
- **Result Storage:** `node.results["ds_raw"] = dataset`: Stores the final fetched dataset (containing the raw averaged I/Q values vs. frequency detuning for each qubit) in the `node.results` dictionary under the key "ds_raw".

## Load Data Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is None)
def load_data(node: QualibrationNode[Parameters, Quam]):
    """Load a previously acquired dataset."""
    load_data_id = node.parameters.load_data_id
    # Load the specified dataset
    node.load_from_id(node.parameters.load_data_id)
    node.parameters.load_data_id = load_data_id
    # Get the active qubits from the loaded node parameters
    node.namespace["qubits"] = get_qubits(node)

### Explanation: Load_data Section

This run action provides an alternative to executing the experiment; it loads data from a previous run identified by an ID.
- **Decorator:** `@node.run_action(skip_if=node.parameters.load_data_id is None)` ensures this action only runs if the `load_data_id` parameter *is* set (i.e., not None).
- **Loading:**
    - `load_data_id = node.parameters.load_data_id`: Stores the ID locally.
    - `node.load_from_id(node.parameters.load_data_id)`: Calls the built-in `QualibrationNode` method to load the parameters and results associated with the specified run ID from the data storage location. This populates `node.parameters` and `node.results` with the loaded data.
    - `node.parameters.load_data_id = load_data_id`: Resets the parameter in the current node instance (loading might overwrite it).
- **Namespace Setup:** `node.namespace["qubits"] = get_qubits(node)`: Ensures that the list of target qubits (needed for analysis and plotting) is correctly set in the namespace based on the *loaded* parameters.

## Analyse Data Section

In [ ]:
@node.run_action(skip_if=node.parameters.simulate)
def analyse_data(node: QualibrationNode[Parameters, Quam]):
    """Analyse the raw data and store the fitted data in another xarray dataset "ds_fit" and the fitted results in the "fit_results" dictionary."""
    node.results["ds_raw"] = process_raw_dataset(node.results["ds_raw"], node)
    node.results["ds_fit"], fit_results = fit_raw_data(node.results["ds_raw"], node)
    node.results["fit_results"] = {k: asdict(v) for k, v in fit_results.items()}

    # Log the relevant information extracted from the data analysis
    log_fitted_results(node.results["fit_results"], log_callable=node.log)
    node.outcomes = {
        qubit_name: ("successful" if fit_result["success"] else "failed")
        for qubit_name, fit_result in node.results["fit_results"].items()
    }

### Explanation: Analyse_data Section

This run action performs post-processing and fitting on the acquired (or loaded) raw data.
- **Decorator:** `@node.run_action(skip_if=node.parameters.simulate)` ensures this action is skipped if the node is only simulating the program.
- **Data Processing:**
    - `node.results["ds_raw"] = process_raw_dataset(node.results["ds_raw"], node)`: Calls an experiment-specific function (`process_raw_dataset`) to perform initial processing on the raw data stored in `node.results["ds_raw"]`. This might involve calculating magnitude and phase from I/Q, unit conversions, or background subtraction. The processed dataset might overwrite the raw one or be stored separately.
- **Fitting:**
    - `node.results["ds_fit"], fit_results = fit_raw_data(node.results["ds_raw"], node)`: Calls another experiment-specific function (`fit_raw_data`) to fit a model (e.g., Lorentzian) to the processed data. This function likely returns:
        - `ds_fit`: An xarray Dataset containing the fitted curves.
        - `fit_results`: A dictionary (or custom objects) containing the fitted parameters (like resonance frequency, linewidth, amplitude, offset) and fit success status for each qubit.
- **Result Storage:**
    - `node.results["ds_fit"] = fitted_ds`: Stores the dataset with the fitted curves.
    - `node.results["fit_results"] = {k: asdict(v) for k, v in fit_results.items()}`: Stores the fitted parameters. `asdict` is used if `fit_results` contains custom dataclass objects, converting them to dictionaries for easier serialization.
- **Logging & Outcomes:**
    - `log_fitted_results(...)`: Calls a function to print or log the key fitted parameters using the QUAlibrate logger.
    - `node.outcomes = {...}`: Creates a dictionary `node.outcomes` summarizing the result for each qubit (e.g., "successful" or "failed") based on the fit success status. This provides a quick summary used by QUAlibrate's reporting and potentially by subsequent nodes in a graph.

## Plot Data Section

In [ ]:
@node.run_action(skip_if=node.parameters.simulate)
def plot_data(node: QualibrationNode[Parameters, Quam]):
    """Plot the raw and fitted data in specific figures whose shape is given by qubit.grid_location."""
    fig_raw_phase = plot_raw_phase(node.results["ds_raw"], node.namespace["qubits"])
    fig_fit_amplitude = plot_raw_amplitude_with_fit(
        node.results["ds_raw"], node.namespace["qubits"], node.results["ds_fit"]
    )
    plt.show()
    # Store the generated figures
    node.results["figures"] = {
        "phase": fig_raw_phase,
        "amplitude": fig_fit_amplitude,
    }

### Explanation: Plot_data Section

This run action generates plots based on the processed and fitted data.
- **Decorator:** `@node.run_action(skip_if=node.parameters.simulate)` skips plotting during simulation.
- **Plot Generation:**
    - Calls experiment-specific plotting functions (e.g., `plot_raw_phase`, `plot_raw_amplitude_with_fit`) imported earlier. These functions typically take the relevant xarray Datasets (`node.results["ds_raw"]`, `node.results["ds_fit"]`) and the list of qubits (`node.namespace["qubits"]`) as input and return Matplotlib figure objects. The plotting functions might use qubit metadata (like `qubit.grid_location`) to arrange subplots logically.
- **Display Plot:** `plt.show()`: Displays the generated plots interactively when the script is run standalone. This line might be commented out for fully automated runs.
- **Store Figures:**
    - `node.results["figures"] = {"phase": fig_raw_phase, "amplitude": fig_fit_amplitude}`: Stores the generated Matplotlib figure objects in the `node.results` dictionary under the key "figures". QUAlibrate automatically saves these figures when `node.save()` is called, making them viewable in the UI.

## Update State Section

In [ ]:
@node.run_action(skip_if=node.parameters.simulate)
def update_state(node: QualibrationNode[Parameters, Quam]):
    """Update the relevant parameters if the qubit data analysis was successful."""
    with node.record_state_updates():
        for q in node.namespace["qubits"]:
            if node.outcomes[q.name] == "failed":
                continue

            q.resonator.f_01 = float(node.results["fit_results"][q.name]["frequency"])
            q.resonator.RF_frequency = float(node.results["fit_results"][q.name]["frequency"])

### Explanation: Update_state Section

This run action proposes changes to the QUAM state based on the results obtained from the analysis.
- **Decorator:** `@node.run_action(skip_if=node.parameters.simulate)` skips state updates during simulation.
- **Context Manager:** `with node.record_state_updates():` is the crucial context manager provided by QUAlibrate for proposing state changes. Any modifications made to QUAM element parameters *inside* this block are tracked.
- **Conditional Update:**
    - `for q in node.namespace["qubits"]:`: Iterates through the target qubits.
    - `if node.outcomes[q.name] == "failed": continue`: Checks the outcome stored during analysis. If the fit failed for a specific qubit, no update is proposed for it.
- **Parameter Modification:**
    - `fitted_freq = node.results["fit_results"][q.name]["frequency"]`: Retrieves the fitted resonance frequency for the current qubit `q`.
    - `q.resonator.f_01 = float(fitted_freq)`: Modifies the `f_01` attribute (often representing the readout frequency or IF) of the `Resonator` object associated with qubit `q`. Because this happens inside the `record_state_updates` block, it's registered as a proposed change.
    - `q.resonator.RF_frequency = float(fitted_freq)`: Similarly updates the RF frequency parameter.
- **Behavior:** When run via QUAlibrate, these proposed changes are presented to the user for approval after the node completes. If run standalone, these changes modify the `node.machine` object in memory but are *not* automatically saved back to the QUAM configuration file unless explicitly programmed to do so outside this block.

## Save Results Section

In [ ]:
@node.run_action()
def save_results(node: QualibrationNode[Parameters, Quam]):
    node.save()

### Explanation: Save_results Section

This final run action is responsible for persisting all the relevant information gathered during the node's execution.
- **Decorator:** `@node.run_action()`: Typically has no `skip_if` condition, ensuring that saving always occurs (unless the node fails catastrophically earlier).
- **Saving Operation:** `node.save()`: Calls the core `save` method of the `QualibrationNode` instance. This method performs several actions:
    - Saves the input parameters (`node.parameters`) used for this specific run.
    - Saves the contents of the `node.results` dictionary, including any data (like xarray Datasets) and Matplotlib figures.
    - Saves the `node.outcomes` dictionary.
    - Saves any state updates proposed within the `node.record_state_updates()` block.
    - Saves metadata like the node name, execution timestamp, and status.
- **Data Location:** The data is saved in a structured directory hierarchy managed by QUAlibrate using the timestamp, unique run id, and node name, allowing for easy retrieval and comparison of results across different runs.